# Libraries

In [1]:
!pip install pmaw

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 188 kB 7.5 MB/s 
     |████████████████████████████████| 54 kB 2.2 MB/s 


In [2]:
import pmaw
import datetime as dt
from datetime import datetime
import pandas as pd
from pmaw import PushshiftAPI

In [3]:
api = PushshiftAPI()

# Subreddit posts downloader

In [ ]:
def start_time(y, m, d):
  start = int(dt.datetime(y, m, d).timestamp())
  return start

def stop_time(y, m, d):
  stop = int(dt.datetime(y, m, d).timestamp())
  return stop

def get_subpost(after, before, title=None, query = None, subreddit = None, limit = None):
  submissions = api.search_submissions(after = after,
                                      before = before,
                                      title = title,
                                      q = query,
                                      subreddit = subreddit,
                                      filter = ['id', 'created_utc', 'author', 
                                                'title', 'subreddit', 'num_comments',
                                                'permalink', 'selftext'],
                                      limit = limit)
  sub_df = pd.DataFrame(submissions)
  sub_df['created_utc'] = pd.to_datetime(sub_df['created_utc'], unit='s')
  sub_df['date'] = sub_df['created_utc'].dt.date
  sub_df['time'] = sub_df['created_utc'].dt.time
  return sub_df

In [ ]:
start = start_time(2022, 1, 1)
stop = stop_time(2022, 3, 31)

In [ ]:
subs = ["wallstreetbets", "investing", "stocks"]

In [ ]:
posts = get_subpost(start, stop, subreddit = subs)
posts = posts.sort_values(by = 'date')

In [ ]:
posts

In [ ]:
dup = posts.duplicated().any()
dup

False

In [ ]:
posts.to_csv('./posts_Jan_Mar_2022.csv', header=True, index=False, columns=list(posts.axes[1]))

# Filter DF

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
df_ticker = pd.read_csv("/content/drive/MyDrive/DATAMAN_2 Project/Codice - work in progess/downloaded_symbol.csv")
tickers = df_ticker['Symbol']

In [53]:
def tickers_filter(tickers, df):
  df_list =[]

  for i in tickers:
    mask = df[df["title"].str.contains(f"{i}")]
    mask = mask.assign(symbol = str(i))
    df_list.append(mask)
  df_filtered = pd.concat(df_list)

  return df_filtered

In [ ]:
filtered = tickers_filter(tickers, posts)
filtered

In [ ]:
filtered.sort_values(by = 'date')

In [ ]:
filtered.to_csv('/content/drive/MyDrive/DATAMAN_2 Project/Jan_Mar_2022_tickers.csv', header=True, index=False, columns=list(filtered.axes[1]))

# EDA

In [ ]:
filtered.info

In [ ]:
filtered = filtered.groupby("symbol")["symbol"].count().reset_index(name= "count").sort_values(['count'], ascending = False)

filtered.head(20)